<H1> Segmenting and Clustering Neighborhoods in Toronto</H1>

This notebook will be used to segment and cluster neighborhoods in Toronto. It will contain 3 main areas:
1. Creating a pandas dataframe containing Postal Code, Borough and Neighborhood by scraping a Wikipedia page
2. Getting the latitude and the longitude coordinates of each neighborhood
3. Exploring and clustering the neighborhoods in Toronto

<H2>1. Creating a pandas dataframe</H2>

In [11]:
import pandas as pd
import numpy as np

As a first step, I will get the data from the Wikipedia page and assign it to a pandas dataframe.

In [37]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' #get URL for dataframe

df=pd.read_html(url, header=0)[0] # transform table into panda dataframe

df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Remove spaces in the column names and rename

In [42]:
df.columns = df.columns.str.strip()

df=df.rename(columns={"Postcode":"Postal Code","Neighbourhood":"Neighborhood"})

df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


Now that the data is in the format of a dataframe, I will clean it in order to make it easer to work with. 

In [44]:
#delete rows with a borough that is Not assigned
df.drop( df[ df['Borough'] == "Not assigned" ].index, inplace=True)

#if a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
df.Neighborhood.replace("Not assigned",df.Borough,inplace=True)

# where more than one neighborhood exists in one postal code area, the two rows are combined into one row with the neighborhoods separated by a comma
df1=df.groupby("Postal Code").agg(lambda x:','.join(set(x))) 

df1.reset_index(inplace=True)
df1.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern,Rouge"
1,M1C,Scarborough,"Rouge Hill,Highland Creek,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Let's see the shape of the dataframe

In [23]:
df1.shape

(103, 3)

<H2>2. Latitude and Longitude</H2>

This section will be used to obtain the latitude and the longitude coordinates of each neighborhood in the above dataframe.

In [45]:
lat_and_long=pd.read_csv('http://cocl.us/Geospatial_data')

lat_and_long.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [46]:
df3 = pd.merge(df1, lat_and_long, on="Postal Code", how='inner')

df3.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill,Highland Creek,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


<H2>3. Exploring Toronto</H2>

This section will be used to explore and cluster the neighborhoods in Toronto.

Importing libraries needed for exploring, clustering and visualization

In [48]:
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         868 KB

The following NEW packages will be INSTALLED:

    altair:  3.2.0-py36_0 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forg

Define Foursquare credentials and version

In [109]:
CLIENT_ID = 'TFE31JBELQWHLJSX5MZTCRPLO2V13XID32IJFAZM03KXUHAG' # your Foursquare ID
CLIENT_SECRET = '5MSLRWWN5FBGL2BNFN2L1T5SE4JYAJOTKB1XSRL05SIO040L' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

I am going to explore the borough of North York

In [57]:
#creating the dataframe for the neighbourhoods in North York
northy_data = df3[df3['Borough'] == 'North York'].reset_index(drop=True)
northy_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M2H,North York,Hillcrest Village,43.803762,-79.363452
1,M2J,North York,"Oriole,Henry Farm,Fairview",43.778517,-79.346556
2,M2K,North York,Bayview Village,43.786947,-79.385975
3,M2L,North York,"York Mills,Silver Hills",43.757490,-79.374714
4,M2M,North York,"Willowdale,Newtonbrook",43.789053,-79.408493


In [58]:
#Getting the geographical coordinates of North York
address = 'North York, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York are 43.7708175, -79.4132998.


Let's visualize the area:

In [59]:
# create map of North York using latitude and longitude values
map_northy = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(northy_data['Latitude'], northy_data['Longitude'], northy_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_northy)  
    
map_northy

Create a function to explore the venues in the neighborhoods of North York

In [50]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
         # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Run the above function on each neighborhood and create a new dataframe called <i>northy_venues</i>

In [69]:
limit=100
northy_venues = getNearbyVenues(names=northy_data['Neighborhood'], latitudes=northy_data['Latitude'], longitudes=northy_data['Longitude'])

Hillcrest Village
Oriole,Henry Farm,Fairview
Bayview Village
York Mills,Silver Hills
Willowdale,Newtonbrook
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Downsview North,Bathurst Manor,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Bedford Park,Lawrence Manor East
Lawrence Manor,Lawrence Heights
Glencairn
North Park,Downsview,Upwood Park
Humber Summit
Humberlea,Emery


In [70]:
print(northy_venues.shape)
northy_venues.head()

(235, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hillcrest Village,43.803762,-79.363452,Eagle's Nest Golf Club,43.805455,-79.364186,Golf Course
1,Hillcrest Village,43.803762,-79.363452,New York Fries,43.803664,-79.363905,Fast Food Restaurant
2,Hillcrest Village,43.803762,-79.363452,AY Jackson Pool,43.804515,-79.366138,Pool
3,Hillcrest Village,43.803762,-79.363452,Villa Madina,43.801685,-79.363938,Mediterranean Restaurant
4,Hillcrest Village,43.803762,-79.363452,Duncan Creek Park,43.805539,-79.360695,Dog Run


Let's visualize how many venues are in each neighbourhood

In [71]:
northy_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",21,21,21,21,21,21
"CFB Toronto,Downsview East",2,2,2,2,2,2
Don Mills North,5,5,5,5,5,5
Downsview Central,2,2,2,2,2,2
"Downsview North,Bathurst Manor,Wilson Heights",20,20,20,20,20,20
Downsview Northwest,4,4,4,4,4,4
Downsview West,5,5,5,5,5,5
"Flemingdon Park,Don Mills South",21,21,21,21,21,21


Let's find out how many unique categories can be curated from all the returned venues

In [73]:
print('There are {} uniques categories.'.format(len(northy_venues['Venue Category'].unique())))

There are 101 uniques categories.


Next, I will analyze each neighbourhood

In [74]:
# one hot encoding
northy_onehot = pd.get_dummies(northy_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
northy_onehot['Neighborhood'] = northy_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [northy_onehot.columns[-1]] + list(northy_onehot.columns[:-1])
northy_onehot = northy_onehot[fixed_columns]

northy_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,...,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [75]:
#visualize the shpae of the new dataframe
northy_onehot.shape

(235, 102)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [77]:
northy_grouped = northy_onehot.groupby('Neighborhood').mean().reset_index()
northy_grouped

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,...,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,Bayview Village,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.250000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
1,"Bedford Park,Lawrence Manor East",0.000000,0.0,0.047619,0.000000,0.000000,0.00,0.000000,0.000000,0.00,...,0.047619,0.000000,0.047619,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
2,"CFB Toronto,Downsview East",0.000000,0.5,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
3,Don Mills North,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
4,Downsview Central,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
5,"Downsview North,Bathurst Manor,Wilson Heights",0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.050000,0.00,...,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.05,0.000000,0.000000,0.000000
6,Downsview Northwest,0.000000,0.0,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
7,Downsview West,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.200000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
8,"Flemingdon Park,Don Mills South",0.000000,0.0,0.000000,0.000000,0.095238,0.00,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
9,Glencairn,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.250000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000


Let's print each neighborhood along with the top 3 most common venues

In [78]:
num_top_venues = 3

for hood in northy_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = northy_grouped[northy_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bayview Village----
                venue  freq
0  Chinese Restaurant  0.25
1                Bank  0.25
2                Café  0.25


----Bedford Park,Lawrence Manor East----
                venue  freq
0         Coffee Shop  0.10
1  Italian Restaurant  0.10
2       Grocery Store  0.05


----CFB Toronto,Downsview East----
       venue  freq
0    Airport   0.5
1       Park   0.5
2  Juice Bar   0.0


----Don Mills North----
                  venue  freq
0  Gym / Fitness Center   0.2
1  Caribbean Restaurant   0.2
2                  Café   0.2


----Downsview Central----
               venue  freq
0         Food Truck   0.5
1     Baseball Field   0.5
2  Accessories Store   0.0


----Downsview North,Bathurst Manor,Wilson Heights----
           venue  freq
0    Coffee Shop  0.10
1  Grocery Store  0.05
2    Supermarket  0.05


----Downsview Northwest----
                venue  freq
0       Grocery Store  0.25
1  Athletics & Sports  0.25
2      Discount Store  0.25


----Downsview West----

Let's put this information into a <i>pandas</i> dataframe

In [80]:
#function to sort the venues in descending order

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [82]:
#create the new dataframe and display the top 10 venues for each neighborhood
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = northy_grouped['Neighborhood']

for ind in np.arange(northy_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(northy_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Bayview Village,Japanese Restaurant,Chinese Restaurant,Café,Bank,Dog Run
1,"Bedford Park,Lawrence Manor East",Italian Restaurant,Coffee Shop,Grocery Store,Sandwich Place,Fast Food Restaurant
2,"CFB Toronto,Downsview East",Airport,Park,Women's Store,Discount Store,Concert Hall
3,Don Mills North,Japanese Restaurant,Gym / Fitness Center,Caribbean Restaurant,Café,Baseball Field
4,Downsview Central,Food Truck,Baseball Field,Women's Store,Discount Store,Concert Hall


<H3> Now we cluster the neighbourhoods </H3>

Run k-means to cluster the neighborhood into 3 clusters.

In [84]:
# set number of clusters
kclusters = 3

northy_grouped_clustering = northy_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(northy_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 2, 0, 1, 0, 0, 0, 0, 0], dtype=int32)

In [90]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

northy_merged = northy_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
northy_merged = northy_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

northy_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M2H,North York,Hillcrest Village,43.803762,-79.363452,0.0,Golf Course,Pool,Fast Food Restaurant,Mediterranean Restaurant,Dog Run
1,M2J,North York,"Oriole,Henry Farm,Fairview",43.778517,-79.346556,0.0,Clothing Store,Fast Food Restaurant,Coffee Shop,Women's Store,Tea Room
2,M2K,North York,Bayview Village,43.786947,-79.385975,0.0,Japanese Restaurant,Chinese Restaurant,Café,Bank,Dog Run
3,M2L,North York,"York Mills,Silver Hills",43.757490,-79.374714,2.0,Park,Women's Store,Discount Store,Concert Hall,Construction & Landscaping
4,M2M,North York,"Willowdale,Newtonbrook",43.789053,-79.408493,NaN,NaN,NaN,NaN,NaN,NaN


Visualize the clusters

In [107]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(northy_merged['Latitude'], northy_merged['Longitude'], northy_merged['Neighborhood'], northy_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow,
        fill=True,
        fill_color=rainbow,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examining clusters

In [100]:
#Cluster 1
northy_merged.loc[northy_merged['Cluster Labels'] == 0, northy_merged.columns[[1] + list(range(3, northy_merged.shape[1]))]]

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,North York,43.803762,-79.363452,0.0,Golf Course,Pool,Fast Food Restaurant,Mediterranean Restaurant,Dog Run
1,North York,43.778517,-79.346556,0.0,Clothing Store,Fast Food Restaurant,Coffee Shop,Women's Store,Tea Room
2,North York,43.786947,-79.385975,0.0,Japanese Restaurant,Chinese Restaurant,Café,Bank,Dog Run
5,North York,43.770120,-79.408493,0.0,Coffee Shop,Ramen Restaurant,Sushi Restaurant,Japanese Restaurant,Restaurant
7,North York,43.782736,-79.442259,0.0,Pizza Place,Home Service,Butcher,Pharmacy,Discount Store
9,North York,43.745906,-79.352188,0.0,Japanese Restaurant,Gym / Fitness Center,Caribbean Restaurant,Café,Baseball Field
10,North York,43.725900,-79.340923,0.0,Coffee Shop,Asian Restaurant,Gym,Beer Store,Discount Store
11,North York,43.754328,-79.442259,0.0,Coffee Shop,Grocery Store,Shopping Mall,Frozen Yogurt Shop,Fast Food Restaurant
12,North York,43.767980,-79.487262,0.0,Coffee Shop,Caribbean Restaurant,Bar,Massage Studio,Discount Store
14,North York,43.739015,-79.506944,0.0,Grocery Store,Park,Hotel,Bank,Shopping Mall


In [101]:
#Cluster 2
northy_merged.loc[northy_merged['Cluster Labels'] == 1, northy_merged.columns[[1] + list(range(3, northy_merged.shape[1]))]]

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
15,North York,43.728496,-79.495697,1.0,Food Truck,Baseball Field,Women's Store,Discount Store,Concert Hall
23,North York,43.724766,-79.532242,1.0,Furniture / Home Store,Baseball Field,Women's Store,Discount Store,Concert Hall


In [102]:
#Cluster 3
northy_merged.loc[northy_merged['Cluster Labels'] == 2, northy_merged.columns[[1] + list(range(3, northy_merged.shape[1]))]]

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
3,North York,43.757490,-79.374714,2.0,Park,Women's Store,Discount Store,Concert Hall,Construction & Landscaping
6,North York,43.752758,-79.400049,2.0,Park,Convenience Store,Bank,Women's Store,Discount Store
8,North York,43.753259,-79.329656,2.0,Park,Food & Drink Shop,Women's Store,Discount Store,Concert Hall
13,North York,43.737473,-79.464763,2.0,Airport,Park,Women's Store,Discount Store,Concert Hall
